In [2]:
from newspaper import Article
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np
import pandas as pd

In [8]:
class SentenceTokenizer(object):
    def __init__(self):
        self.kkma = Kkma()
        self.twitter = Twitter()
        f = open("preprocessing.txt",encoding='UTF-8')
        preprocessing_Text = f.read()
        stop_words = word_tokenize(preprocessing_Text)
        self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자"
,"아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",]
    def url2sentences(self, url):
        article = Article(url, language='ko')
        article.download()
        article.parse()
        sentences = self.kkma.sentences(article.text)
        
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence))
                    if noun not in self.stopwords and len(noun) > 1]))
        return nouns
    def preprocess_sentence(sentence, remove_stopwords = True):
        sentence = sentence.upper() # 텍스트 대문자화
        sentence = BeautifulSoup(sentence, "lxml").text # <br />, <a href = ...> 등의 html 태그 제거
        sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열  제거 Ex) my husband (and myself) for => my husband for
        sentence = re.sub('"','', sentence) # 쌍따옴표 " 제거
        sentence = re.sub('[ㅋ]{2,}', 'ㅋㅋ', sentence) # ㅋ이 3개 이상이면 2개로 변경. Ex) ㅋㅋㅋㅋㅋㅋ -> ㅋㅋ

        # 불용어 제거 (Text)
        if remove_stopwords:
            tokens = ' '.join(word for word in sentence.split() if not word in stop_words if len(word) > 1)
        # 불용어 미제거 (Summary)
        else:
            tokens = ' '.join(word for word in sentence.split() if len(word) > 1)
        return tokens

In [9]:
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer()
        self.cnt_vec = CountVectorizer()
        self.graph_sentence = []
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    def build_words_graph(self, sentence):
        cnt_vec_mat = normalize(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

In [10]:
class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
            
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

In [11]:
class TextRank(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        
        if text[:5] in ('http:', 'https'):
            self.sentences = self.sent_tokenize.url2sentences(text)
        else:
            self.sentences = self.sent_tokenize.text2sentences(text)
        
        
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        return summary
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        
        return keywords

In [12]:
url = 'https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=100&oid=025&aid=0003109159'
textrank = TextRank(url)
a = textrank.summarize(6)
for row in a:
    print(row)
    print()
print(a)
print("".join(a))
print('keywords :',textrank.keywords())

국민의 힘의 새 지도부를 선출하는 전당대회가 11일 서울 여의도 중앙 당사에서 열린 가운데 이 준석, 나 경원 당대표 후보가 결과 발표를 지켜보고 있다.

오종 택 기자 국민의 힘의 새 지도부를 선출하는 전당대회가 11일 서울 여의도 중앙 당사에서 열린 가운데 이 준석, 나 경원 당대표 후보가 결과 발표를 지켜보고 있다.

오종 택 기자 국민의 힘 대표 후보자 나 경원, 이 준석, 조경 태, 주호 영, 홍문 표 국민의 힘 대표 후보자들이 11일 오전 서울 영등포구 국민의 힘 당사에서 열린 전당대회에서 박수치고 있다.

오종 택 기자 국민의 힘 대표 후보자 나 경원, 이 준석, 조경 태, 주호 영, 홍문 표 국민의 힘 대표 후보자들이 11일 오전 서울 영등포구 국민의 힘 당사에서 열린 전당대회에서 박수치고 있다.

이날 이 준석 후보가 대표로 선출됐다.

국민의 힘 나 경원 전 의원은 이날 발표된 대표 선거 결과 당원투표와 국민 여론조사를 합쳐 37.1% 득표율로 이 대표 (43.8% )에게 6.7% 포인트 차이로 밀려 2위를 기록했다.

['국민의 힘의 새 지도부를 선출하는 전당대회가 11일 서울 여의도 중앙 당사에서 열린 가운데 이 준석, 나 경원 당대표 후보가 결과 발표를 지켜보고 있다.', '오종 택 기자 국민의 힘의 새 지도부를 선출하는 전당대회가 11일 서울 여의도 중앙 당사에서 열린 가운데 이 준석, 나 경원 당대표 후보가 결과 발표를 지켜보고 있다.', '오종 택 기자 국민의 힘 대표 후보자 나 경원, 이 준석, 조경 태, 주호 영, 홍문 표 국민의 힘 대표 후보자들이 11일 오전 서울 영등포구 국민의 힘 당사에서 열린 전당대회에서 박수치고 있다.', '오종 택 기자 국민의 힘 대표 후보자 나 경원, 이 준석, 조경 태, 주호 영, 홍문 표 국민의 힘 대표 후보자들이 11일 오전 서울 영등포구 국민의 힘 당사에서 열린 전당대회에서 박수치고 있다.', '이날 이 준석 후보가 대표로 선출됐다.', '국민의 힘 나 경원 전 의원은 이날 발표된 대표 선거

In [13]:
print(textrank.sent_graph)

[[ 1.         -0.19785704 -0.08993434 -0.12026211 -0.08993434 -0.12026211
  -0.08529334 -0.07103962 -0.0217275  -0.13229689 -0.09726874 -0.02604108
  -0.         -0.06148607 -0.04340486 -0.0419783  -0.         -0.
  -0.0175859  -0.05444659 -0.0266958  -0.04334719 -0.09041833 -0.
  -0.425     ]
 [-0.19785704  1.         -0.08993434 -0.12026211 -0.08993434 -0.12026211
  -0.08529334 -0.07103962 -0.0217275  -0.13229689 -0.09726874 -0.02604108
  -0.         -0.06148607 -0.04340486 -0.0419783  -0.         -0.
  -0.0175859  -0.05444659 -0.0266958  -0.04334719 -0.09041833 -0.
  -0.425     ]
 [-0.06977122 -0.06977122  1.         -0.04225283 -0.25503556 -0.04225283
  -0.02592351 -0.07215884 -0.03212308 -0.06860976 -0.04330831 -0.0385005
  -0.         -0.06040577 -0.03208602 -0.04287431 -0.         -0.0453844
  -0.02599992 -0.0804967  -0.08831187 -0.03204338 -0.         -0.
  -0.        ]
 [-0.09077441 -0.09077441 -0.04110926  1.         -0.04110926 -0.2621301
  -0.18591018 -0.02517314 -0.0473584

In [21]:
print(len(textrank.words_graph))

131


In [15]:
print(textrank.sorted_sent_rank_idx)

[0, 1, 7, 4, 2, 5, 3, 9, 6, 8, 17, 20, 23, 19, 21, 10, 15, 22, 18, 14, 16, 11, 13, 12, 24]


In [16]:
print(textrank.sent_rank_idx)

{0: 2.0536261880153375, 1: 2.053626188015337, 2: 1.600696318749654, 3: 1.5245599813792872, 4: 1.6006963187496541, 5: 1.5245599813792874, 6: 1.0251142756360851, 7: 1.8879055449104662, 8: 0.9528999697544305, 9: 1.3015418568958959, 10: 0.6617301375321218, 11: 0.600877767082211, 12: 0.37957019503523914, 13: 0.5922160098068282, 14: 0.6255728180877963, 15: 0.6575947429581052, 16: 0.6076783057332382, 17: 0.8997380610964386, 18: 0.6422341078485275, 19: 0.6989854674275725, 20: 0.8315885213738615, 21: 0.6640563821789149, 22: 0.6465607816099007, 23: 0.7351554839553937, 24: 0.231214594788418}


In [17]:
vect = CountVectorizer(max_features=10000, max_df=.15)
X = vect.fit_transform(textrank.sentences)

In [18]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=10,
                               lear                                                                                          ning_method="batch", max_iter=25, random_state=0)
document_topics = lda.fit_transform(X)

SyntaxError: invalid syntax (<ipython-input-18-d54524fabaad>, line 3)